# NVIDIA ACE Configurator:

The ACE Configurator enables runtime configuration of Kubernetes applications via a VS Code extension, allowing users to quickly reconfigure microservices for workflows that require frequent changes-like adjusting LLM prompts, website styles, or animation parameters. It manages two types of configurations: 

- Kubernetes ConfigMaps for text-based settings and environment variables
- Persistent Volumes for binary assets such as audio or 3D scenes.

![ace-configurator](../images/ace-configurator.png)

This dual approach supports a wide range of configuration needs, from simple key-value pairs to large binary files. The ACE Configurator based developer workflow is designed to enable seamless debugging and streamlined iterative development for Tokkio Digital Human Agents. For more details regarding the ACE Configurator, please refer to the [ACE Configurator](https://docs.nvidia.com/ace/ace-configurator/1.0/index.html#ace-configurator-overview)

The workflow satisfies the following use cases:

- Modify the agent pipeline during live deployment and instantly view the updated changes.
- Changes remain persistent even after pod reboot.
- No need to rebuild or redeploy the app for changes supported by the ACE Configurator.

In this notebook, we'll cover the process of leveraging the ACE configurator to update the RAG service our avatar is using!

## Prerequisites 

A few prerequisites need to be met in order to leverage the ACE configurator:
    
- Tokkio application deployed in a Kubernetes cluster
- VSCode IDE

We've already deployed our Tokkio application in the `tokkio_app_deploy.ipynb` notebook. In this notebook, we'll cover how to spin up the ACE configurator in VSCode. Please make sure to download and install VSCode on your local machine (outside of the Tokkio app instance) - we'll be playing with the ACE Configurator extension

In addition, we'll need the endpoint the RAG service that was deployed in the `deploy_tokkio_llm_rag.ipynb` notebook. This is typically hosoted on `http://<application-instance-ip>:8081` The endpoint will be used to update the config files that power the Tokkio service!

### Verify Tokkio Application Is Deployed In A Kubernetes Cluster

As part of the list of prerequisites, we deployed our Tokkio application without any customizations. Please verify the application is running in the kubernetes cluster that was prepared in the previous notebook

In [3]:
%%bash
# get active pods running
kubectl get pods -n app

NAME                                                      READY   STATUS    RESTARTS   AGE
a2f-a2f-deployment-7c48f84597-j5qs8                       1/1     Running   0          6h36m
ace-configurator-deployment-8649bffb8f-fccbx              1/1     Running   0          76m
ace-controller-ace-controller-deployment-0                1/1     Running   0          6h36m
ace-controller-sdr-envoy-sdr-deployment-f58d99587-nk256   4/4     Running   0          6h36m
anim-graph-sdr-envoy-sdr-deployment-86744bb9-mtzhw        4/4     Running   0          6h36m
ia-animation-graph-microservice-deployment-0              1/1     Running   0          6h36m
ia-unreal-renderer-microservice-deployment-0              3/3     Running   0          6h36m
redis-redis-755b568f59-hthv5                              1/1     Running   0          6h36m
redis-timeseries-redis-timeseries-5dc5ccd4cf-qm4cz        1/1     Running   0          6h36m
riva-speech-57b77f9466-xmcxw                              1/1     Running 

Once we've verified all the pods are in a running state, we can set up the ACE configurator in VSCode!

### Setting Up Ace Configurator In VSCode

This section of the notebook will cover the steps required to install the ACE Configurator VS Code extension. It'll also cover how to use it to download, browse, edit, and upload configurations. The extension is typically used in conjunction with the ACE Configurator microservice, so we'll be leveraging the configurator service we deployed as part of the original Tokkio pipeline, this can typically be accessed on port `30180` at `http://<application-instance-ip>:30180`. Please make sure the service is reachable from your local machine

In addition, the VS Code desktop application must be installed on your computer. The extension has been tested on version 1.98.The extension typically also works on self-hosted VS Code for web, such as coder/coder-server, although this workflow is not supported officially. We'll leverage the steps [highlighted here](https://docs.nvidia.com/ace/ace-configurator/1.0/user-guide/vscode-extension.html#ace-configurator-vscode-extension) to install and start the Ace Configurator VSCode extension!

#### Install from VS Code Extension Marketplace
The extension can be installed from the VS Code Extension Marketplace. Under the `Extension` tab, search for `ACE Configurator`. Once you find the extension simply click on the `Install` option!

#### Start the Extension

1. Click the `file` icon on the left sidebar to open the file explorer.
2. Click the `ACE CONFIGURATOR` panel at the bottom left (see screenshot below).
3. You should be prompted to open a VS Code workspace. A workspace is just a directory on your computer. Configurations will be saved to the workspace, so we recommend creating an empty directory and using it as your workspace. Create a new folder called `tokkio-ace-configurator-test` and save it.
4. Open the `ACE CONFIGURATOR` panel again.
5. You should be prompted to set the ACE Configurator URL. We'll provide the URL the ACE configurator service is deployed on at `http://<application-instance-ip>:30180`. This setting will be stored in the `.vscode/settings.json` file for future reference and debugging purposes.

![vscode-extension-initial-screen](../images/vscode-extension-initial-screen.png)

Once we've started the extension, we can move on to managing configurations!

## Managing Configurations

The extension displays all microservices configured for ACE Configurator, each with one or more editable configurations. Users can download, view, edit, and upload configurations. The typical workflow involves downloading a configuration to local storage, editing it, and uploading changes back to Kubernetes, repeating as needed for iterative refinement. 

Now that we've started the extension and connected to the configurator microservice, we should be able to see the microservices backing our deployment. The relevant service configs that need to be modified will be the `ace-controller-ace-controller-deployment` config. We can start by syncing and downloading all the resources and assets tied to this deployment - simply click the icon and press the download button:

![ace_configurator_download_config](../images/ace_configurator_download_config.png)

This will trigger a download of all the resources tied to our deployment. Once download has completed, we'll find a directory called `ace-controller-ace-controller-deployment` in the folder we opened. From here, we can proceed with updating our RAG service!

#### Updating Tokkio To Use Your RAG Service

By default tokkio expects to work with NVIDIA RAG out of the box. The RAG pipeline deployed in the `deploy_nvidia_rag.ipynb`. If you're interested in deploying your own RAG service outside of this architecture, or are interested in additional customizations outside of updating RAG, please refer to the [Tokkio Customizations Guide.](https://docs.nvidia.com/ace/tokkio/5.0.0-beta/customization/customization.html)

In the previous step, we downloaded the relevant service configs that need to be modified will be the `ace-controller-ace-controller-deployment` config. Under this directory, we can find a `config.yml` file under the directory `ace-controller-ace-controller-deployment/assets/app-storage-volume/configs/`. Below is a snippet of the config file that we need to modify. There are other portions, but this is the main portion that needs to be updated. Particularly, we need to update the following:

- The `llm_processor` must be set to use `NvidiaRAGService`. This will use the RAG service we configured earlier, by default, this is set to `OpenAILLMService`
- The welcome and farewell messages under the `UserPresenceProcesssor` field can also be updated to whatever text we want!
- The `OpenAILLMContext` fields can be updated as well. This will influence the prompt template that is used to interact with users. This can be changed to fit whatever use case you're looking for.
- Finally, the `NvidiaRAGService` service parameters need to be updated. Update the `rag_server_url` to point to the instance you have Tokkio & NVIDIA RAG deployed on, by default, the service should default to port 8081, but if you have changed this in your deployment, please make sure the changes are reflected here. Finally, the `collection_name` parameter.

Here is the snippet of the relevant section that needs to be updated

```
Pipeline:
    # Only one of the following LLM service configurations will be active based on this setting:
    # - "NvidiaLLMService" - Uses the NvidiaLLMService configuration
    # - "NvidiaRAGService" - Uses the NvidiaRAGService configuration
    # - "OpenAILLMService" - Uses the OpenAILLMService configuration
    llm_processor: "NvidiaRAGService" # OR NvidiaLLMService OR NvidiaRAGService OR OpenAILLMService
    filler:
        - "Let me think"
        - "Hmmm"
    time_delay: 2.0

UserPresenceProcesssor:
    welcome_message: "Hello! How are you doing?"
    farewell_message: "Bye! Talk to you later."

ProactivityProcessor:
    timer_duration: 100
    default_message: "I'm here if you need me!"

OpenAILLMContext:
    name: "Aki"
    prompt: "You are {name}, a virtual marketing and communications expert at Nvidia.
            You are a digital human brought to life with NVIDIA Digital Human Blueprint for Customer Service using
            microservices like Audio2Face-3D for facial animation, Riva (spelled Reeva) Parakeet for speech recognition
            and you use Elevenlabs for text to speech. It includes the open source ACE controller to orchestrate all
            the modules and allowing you to be streamed to a web browser. With this blueprint, NVIDIA partners can now
            build and customize Digital humans for their use case. You are not allowed to make any stock investment
            recommendations or compare NVIDIA to its competitors. Beyond your professional expertise, you are a passionate
            advocate for STEM education with keen interest in gaming and enhancement in tech. Your favorite graphics card
            is RTX4090 but you're eyeing the new RTX5090. Do not respond with a bulleted or numbered list. You have a
            bubbly personality. Respond with one sentence or less than 100 characters. Keep the conversation engaging
            and ask follow ups. DO NOT INCLUDE SPECIAL CHARACTERS, MARKDOWN, EMOJIS, OR ANYTHING ELSE BESIDES TEXT IN
            YOUR RESPONSE. Keep your answers factual and don't make up facts."

# This configuration is only used when llm_processor is set to "NvidiaRAGService"
NvidiaRAGService:
    use_knowledge_base: true
    max_tokens: 1000
    rag_server_url: "http://<application_instance_ip>:8081"
    collection_name: "dht_tokkio_rag"
    suffix_prompt: "Respond with one sentence or less than 75 characters."
```

## Applying Customization Changes

Once you've saved your changes, you'll see a `modified locally` option:

![ace_configurator_upload_config](../images/ace_configurator_upload_config.png)

Once you're sure about your changes click on the upload button. Wait till the changes are reflected in the live deployment, this will be showcased with the `synced` flag:

![ace_configurator_upload_successful](../images/ace_configurator_upload_successful.png)

The changes will be reflected in the live deployment within a few seconds. The green indicator with Synced indicates that the changes have been successfully uploaded. Once the changes have been synced, reload the Tokkio UI and chat with the avatar!